In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wilcoxon
from statannotations.Annotator import Annotator
import sys
sys.path.append('/home/local/VANDERBILT/litz/github/MASILab/lobe_seg')
from luna16_preprocess import get_kfolds

In [7]:
johof_dir = "/home/local/VANDERBILT/litz/data/imagevu/nifti/active_learning/dataset_al/metrics_johof.csv"
johof_df = pd.read_csv(johof_dir)
johof_df

,Unnamed: 0,0,1,2,3,4,input_path
0,0,0.962879,0.957947,0.935841,0.897471,0.969076,/home/local/VANDERBILT/litz/data/imagevu/nifti...
1,1,0.923543,0.924146,0.937117,0.910687,0.971423,/home/local/VANDERBILT/litz/data/imagevu/nifti...
2,2,0.970749,0.968505,0.840396,0.073641,0.779342,/home/local/VANDERBILT/litz/data/imagevu/nifti...
3,3,0.990082,0.986234,0.968397,0.872367,0.977214,/home/local/VANDERBILT/litz/data/imagevu/nifti...
4,4,0.973989,0.972799,0.954973,0.915483,0.972448,/home/local/VANDERBILT/litz/data/imagevu/nifti...
...,...,...,...,...,...,...,...
83,83,0.976401,0.966447,0.945534,0.910667,0.916076,/home/local/VANDERBILT/litz/data/imagevu/nifti...
84,84,0.978435,0.972429,0.972693,0.890979,0.943625,/home/local/VANDERBILT/litz/data/imagevu/nifti...
85,85,0.992220,0.987489,0.975370,0.904444,0.980433,/home/local/VANDERBILT/litz/data/imagevu/nifti...
86,86,0.952160,0.936398,0.954205,0.912543,0.953439,/home/local/VANDERBILT/litz/data/imagevu/nifti...


In [8]:
al_model_dir = "/home/local/VANDERBILT/litz/github/MASILab/lobe_seg/models/0823cval"
kfolds_path = "/home/local/VANDERBILT/litz/data/luna16/5fold_qa.csv"
# kfolds_path = "/home/local/VANDERBILT/litz/data/imagevu/nifti/active_learning/dataset_rand/5folds.csv"
fold_paths = get_kfolds(kfolds_path)
al_folds = []
for d, _, _ in os.walk(al_model_dir):
    al_folds.extend(glob.glob(os.path.join(d, "al_test.csv")))
al_folds = sorted(al_folds)
al_df = pd.read_csv(al_folds[0])
al_df["fold"] = int(al_folds[0][-13])
for i, al_fold in enumerate(al_folds):
    if i > 0:
        al_fold_df = pd.read_csv(al_fold)
        al_fold_df["fold"] = int(al_fold[-13])
        al_df = pd.concat([al_df, al_fold_df], ignore_index=True)
al_df.drop(columns=['Unnamed: 0'], inplace=True)
al_df

,0,1,2,3,4,input_path,fold
0,0.969056,0.969789,0.928927,0.735774,0.925145,/home/local/VANDERBILT/litz/data/imagevu/nifti...,1
1,0.974223,0.967472,0.943733,0.900645,0.956758,/home/local/VANDERBILT/litz/data/imagevu/nifti...,1
2,0.971829,0.971233,0.935199,0.889055,0.967110,/home/local/VANDERBILT/litz/data/imagevu/nifti...,1
3,0.940524,0.943146,0.941654,0.830499,0.957800,/home/local/VANDERBILT/litz/data/imagevu/nifti...,1
4,0.980030,0.974781,0.956007,0.881633,0.925915,/home/local/VANDERBILT/litz/data/imagevu/nifti...,1
...,...,...,...,...,...,...,...
83,0.977917,0.973495,0.966466,0.950268,0.974695,/home/local/VANDERBILT/litz/data/imagevu/nifti...,5
84,0.983842,0.984236,0.970112,0.910186,0.986022,/home/local/VANDERBILT/litz/data/imagevu/nifti...,5
85,0.989926,0.988232,0.983703,0.966125,0.988674,/home/local/VANDERBILT/litz/data/imagevu/nifti...,5
86,0.913412,0.892569,0.957600,0.947319,0.948422,/home/local/VANDERBILT/litz/data/imagevu/nifti...,5


In [10]:
def lobe_map(x):
    if x=='0': return 'LUL'
    if x=='1': return 'LLL'
    if x=='2': return 'RUL'
    if x=='3': return 'RML'
    if x=='4': return 'RLL'

johof_long = pd.melt(johof_df, id_vars=['input_path'], value_vars=['0', '1', '2', '3', '4'])
johof_long['model'] = 'Hof.'
al_long = pd.melt(al_df, id_vars=['input_path'], value_vars=['0', '1', '2', '3', '4'])
al_long['model'] = 'Proposed'
all_df = pd.concat([johof_long, al_long], ignore_index=True)
# # convert lobe numbers to names
all_df = all_df.rename(columns={'variable':'Lobe', 'value': 'Dice score'})
all_df['Lobe'] = all_df['Lobe'].apply(lobe_map)

In [11]:
all_df

,input_path,Lobe,Dice score,model
0,/home/local/VANDERBILT/litz/data/imagevu/nifti...,LUL,0.962879,Hof.
1,/home/local/VANDERBILT/litz/data/imagevu/nifti...,LUL,0.923543,Hof.
2,/home/local/VANDERBILT/litz/data/imagevu/nifti...,LUL,0.970749,Hof.
3,/home/local/VANDERBILT/litz/data/imagevu/nifti...,LUL,0.990082,Hof.
4,/home/local/VANDERBILT/litz/data/imagevu/nifti...,LUL,0.973989,Hof.
...,...,...,...,...
875,/home/local/VANDERBILT/litz/data/imagevu/nifti...,RLL,0.974695,Proposed
876,/home/local/VANDERBILT/litz/data/imagevu/nifti...,RLL,0.986022,Proposed
877,/home/local/VANDERBILT/litz/data/imagevu/nifti...,RLL,0.988674,Proposed
878,/home/local/VANDERBILT/litz/data/imagevu/nifti...,RLL,0.948422,Proposed
